In [ ]:
import os 
import numpy as np

In [ ]:
is_init = False
print(os.listdir('Data'))
for dir in os.listdir('Data'):
	for i in os.listdir('Data/'+dir):
		if(i.split('.')[-1] == 'npy'):
			if(not is_init):
				is_init = True
				X = np.load('Data/' + dir + '/' +i)
				size = X.shape[0]
				y = np.array([dir]*size) 
			else:
				x = np.load('Data/' + dir + '/' +i)
				size = x.shape[0]
				X = np.concatenate((X , x),axis=0)
				y = np.concatenate((y,np.array([(dir)]*size)),axis=0)
print(X.shape,y.shape)

In [ ]:
print(y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()
y = lc.fit_transform(y)
labels = lc.classes_
print(labels)
print(y)

In [ ]:
# one hot encode y
from keras.utils import to_categorical
y = to_categorical(y)
print(y.shape)

In [ ]:
num_rows = X.shape[0]
shuffled_indices = np.random.permutation(num_rows)
X = X[shuffled_indices]
y = y[shuffled_indices]
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Dropout

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.5, min_lr=0.00001)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping, lr_reduction])

In [ ]:
y_pred = model.predict(X_test)
y_temp = np.zeros([y_test.shape[0] , y_test.shape[1]] , dtype=int)
for i in range(y_pred.shape[0]):
	y_temp[i][np.argmax(y_pred[i])] = 1
y_pred = y_temp

In [ ]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
print("Classification Report")
print(classification_report(y_test, y_pred, target_names=labels))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the confusion matrix
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

# Create a heatmap with labels
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)

# Set the title and labels
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

# Display the plot
plt.show()

In [ ]:
model.save('Models/model.keras')
np.save('Models/labels.npy',labels)

In [ ]:
print(X.shape , X_train.shape , X_test.shape)